In [7]:
import tweepy

bearer_token = "AAAAAAAAAAAAAAAAAAAAAAhrjgEAAAAAtaINtJuJDU%2F%2BCdOGL3BFon0rYzo%3DRBGMBkpmeImGMYSEnM0Sa8F2mCwZlidarulMjtMOuKqla8aWCs" # Twitter API
client = tweepy.Client(bearer_token=bearer_token) # Twitter API v2 with OAuth2.0 Version

#let's request and paginate the tweets
query = 'context:152.825047692124442624 -is:retweet lang:en'

limit = 1000 #n of tweets to be requested


tweets = tweepy.Paginator(client.search_recent_tweets,
                          query=query,
                          expansions=['author_id'],
                          tweet_fields=['public_metrics'],
                          max_results=100).flatten(limit=limit)

In [8]:
#Let's create the panda dataframe
import pandas as pd

df_tweets = pd.DataFrame([tweet.data['text'],
                          tweet.data['public_metrics']['retweet_count'],
                          tweet.data['public_metrics']['like_count'],
                          tweet.data['author_id']
                         ] for tweet in tweets) # convert to pandas df

In [9]:
display(df_tweets)

,0,1,2,3
0,Lunch date with one of my princesses after her...,0,0,40560852
1,Jim: So we’re still on for lunch? You’re meeti...,0,0,1391308289720258560
2,lunch and dinner is when you proteinmax,0,0,1276340518939766786
3,yum yum lunch so good,0,0,956004288517599232
4,The best part about German dinners is that the...,0,0,277589619
...,...,...,...,...
995,"Thai Curried Winter Squash with Green Beans, C...",0,0,1605972011913580547
996,@YouTube @firstwefeast I think @firstwefeast ...,0,0,1543336965440929796
997,Lunch is served! Quaker Steaks is serving hot ...,0,2,30863863
998,Chanyeol sent a food truck support to Suho!! 🥰...,0,0,3113649555


In [10]:
#Let's rename the columns
df_tweets.rename(columns={0: "Text", 1: "Retweet", 2: "Likes", 3: "user_id"}, inplace=True)

In [11]:
#getting the number of followers(we need to do a for loop because we can ask for up to 100 followers at the time)
user_list = list(df_tweets["user_id"])
followers = pd.DataFrame()
#print(user_list[(1-1)*100:1*100-1])
for i in range(1,limit//100):
    users = tweepy.Paginator(client.get_users, 
                                 ids=user_list[(i-1)*100:i*100],
                                 user_fields=['public_metrics']).flatten(limit=100)
    f = pd.DataFrame([user.data['public_metrics']['followers_count'] for user in users])
    display(f)
    followers = pd.concat([followers,f])
display(followers)



    
    
    
    
    
#Let's add the column for the followers
#df_tweets['followers'] = followers

Unexpected parameter: pagination_token


,0
0,1872
1,85
2,128
3,51
4,807
...,...
94,34
95,1547
96,1209
97,2202


Unexpected parameter: pagination_token


,0
0,82
1,2
2,354
3,22
4,120
...,...
94,60393
95,29
96,3579
97,541


Unexpected parameter: pagination_token


...

Unexpected parameter: pagination_token


,0
0,46
1,155
2,1308
3,4991
4,5327
...,...
94,1131
95,595
96,1
97,7762


Unexpected parameter: pagination_token


,0
0,262
1,176
2,17319
3,4826
4,61
...,...
94,7
95,4687
96,159
97,81059


,0
0,1872
1,85
2,128
3,51
4,807
...,...
94,7
95,4687
96,159
97,81059


In [14]:
#getting the number of followers(we need to do a for loop because we can ask for up to 100 followers at the time)
user_list = list(df_tweets["user_id"])
followers = []
for i in range(1,limit//100):
    users = tweepy.Paginator(client.get_users, 
                                 ids=user_list[(i-1)*100:i*100],
                                 user_fields=['public_metrics']).flatten(limit=100)
    f = [user.data['public_metrics']['followers_count'] for user in users]
    followers.append(f)
    
print(followers)
    



    
    
    
    
    
#Let's add the column for the followers
#df_tweets['followers'] = followers

Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token
Unexpected parameter: pagination_token


[[1872, 85, 128, 51, 807, 762, 2235, 752, 5176, 139, 4040, 299, 96, 322, 12628, 856, 25, 741, 2012, 231, 1571, 112, 4140, 17, 361, 193, 0, 328, 427, 350, 36920, 14567, 1110472, 2992, 48, 2500, 16, 964, 1830, 250, 119, 492, 828, 815, 140, 52, 406, 763, 36, 813, 658, 2189, 448, 530, 234, 216, 5355, 1003, 108, 40, 539, 120, 620, 198, 1228, 2991, 2643, 317, 426, 5000, 2951, 0, 33, 103, 602382, 80, 11, 3106, 202, 2927, 1440, 145, 22, 124, 11611, 499, 197, 221, 267, 275, 72, 376, 482, 228, 34, 1547, 1209, 2202, 576, 1538], [82, 2, 354, 22, 120, 1098, 938, 176, 2282, 1014, 22, 31, 22, 1415, 383, 267, 2765, 12, 42, 1242, 350, 3426, 409, 981, 50, 1195, 131081, 250, 1890, 59, 650, 484, 5440, 5113, 628, 189, 34537, 180, 1515, 328, 6660, 429, 9062, 0, 37, 247, 705, 4164, 769, 283744, 837, 38, 13, 45, 10770, 746, 773, 274, 4041, 166, 4293, 144, 248, 37, 210, 121384, 49, 3, 140, 734, 245, 1286, 11611, 4374, 274, 66, 21, 194, 18, 3561, 297, 1339, 214, 199, 625, 281, 4868, 29268, 37, 50, 2098, 792, 21

In [ ]:
display(df_tweets)

In [ ]:
import os

path = os.getcwd()

In [ ]:
_ = df_tweets.to_csv(os.path.join(path,'Dataset1.csv'))#creates a csv file containing the tweets